In [1]:
import pandas as pd
from pyfaidx import Fasta
import random
from tqdm import tqdm
import os
import csv

# ================= 1. CẤU HÌNH HỆ THỐNG (QUAN TRỌNG) =================
# Hãy đảm bảo đường dẫn file chính xác
GTF_FILE = r'D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\gencode.v49.basic.annotation.gtf'      # File GTF
FASTA_FILE = r'D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\Homo_sapiens.GRCh38.dna.primary_assembly.fa'   # File FASTA
OUTPUT_FILE = r'D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv'       # File kết quả (~9-10GB)

# Cấu hình Sequence
WINDOW = 300                # Bán kính 200bp
SEQ_LEN = (WINDOW * 2) + 1  # Tổng độ dài 401bp (Tâm tại index 200)
WRITE_BATCH_SIZE = 10000    # Ghi đĩa mỗi 10k dòng để tiết kiệm RAM

# Cấu hình Tỉ lệ (Ratio)
# Bạn muốn 100 Negative : 1 Donor : 1 Acceptor
# Tức là Negative = 50 lần (Donor + Acceptor)
NEGATIVE_MULTIPLIER = 50 

# Danh sách nhiễm sắc thể chuẩn (Bỏ qua các mảnh contig rác để data sạch)
VALID_CHROMS = set([str(i) for i in range(1, 23)] + ['X', 'Y', 'M', 'MT'])

# ================= 2. CÁC HÀM XỬ LÝ CỐT LÕI =================

def normalize_chrom(chrom_name):
    """
    Chuẩn hóa tên nhiễm sắc thể để khớp giữa GTF và FASTA.
    Giải quyết vấn đề: 'chr1' vs '1', 'NC_000001' vs '1'.
    """
    s = str(chrom_name).strip()
    
    # Xử lý dạng RefSeq (NC_...)
    if s.startswith("NC_"):
        try:
            val = int(s.split('_')[1].split('.')[0])
            if val == 23: return 'X'
            if val == 24: return 'Y'
            if val == 12920: return 'M'
            return str(val)
        except: return s
        
    # Xử lý dạng UCSC (chr...)
    if s.lower().startswith('chr'):
        raw = s[3:]
        if raw == 'M': return 'M'
        return raw
        
    return s

def get_sequence_strict(genome, raw_chrom, center_pos, strand, window):
    """
    Trích xuất sequence tại tâm (1-based).
    Yêu cầu: Không padding, không 'N', đảo mạch nếu là Strand (-).
    """
    norm_chrom = normalize_chrom(raw_chrom)
    
    # Tìm key tương ứng trong file FASTA
    fasta_key = None
    if norm_chrom in genome: fasta_key = norm_chrom
    elif f"chr{norm_chrom}" in genome: fasta_key = f"chr{norm_chrom}"
    elif norm_chrom == 'M' and 'chrM' in genome: fasta_key = 'chrM'
    
    if fasta_key is None: return None # Không tìm thấy chromosome

    # Tính tọa độ 0-based
    start_idx = center_pos - 1 - window
    end_idx = center_pos + window
    
    # Check biên giới
    if start_idx < 0 or end_idx >= len(genome[fasta_key]): return None
        
    try:
        seq_obj = genome[fasta_key][start_idx : end_idx]
        seq = seq_obj.seq.upper()
        
        # Check chất lượng
        if len(seq) != (window * 2 + 1): return None
        if 'N' in seq: return None # Loại bỏ sequence chứa N

        # Đảo mạch (Reverse Complement) cho Strand (-)
        if strand == '-':
            mapping = str.maketrans("ATCG", "TAGC")
            seq = seq.translate(mapping)[::-1]
            
        return seq
    except: return None

In [2]:
# ================= 3. CHƯƠNG TRÌNH CHÍNH (STREAMING) =================

print(f"--- GENERATING MASSIVE DATASET (Ratio 100:1:1) ---")
print(f"Target: ~200k Donor, ~200k Acceptor, ~20M Negatives")

# 1. Load Resources
if not os.path.exists(FASTA_FILE):
    print(f"LỖI: Không tìm thấy {FASTA_FILE}"); 
if not os.path.exists(GTF_FILE):
    print(f"LỖI: Không tìm thấy {GTF_FILE}"); 
print("[1/4] Loading Genome (Lazy Load)...")
genome = Fasta(FASTA_FILE, sequence_always_upper=True)

--- GENERATING MASSIVE DATASET (Ratio 100:1:1) ---
Target: ~200k Donor, ~200k Acceptor, ~20M Negatives
[1/4] Loading Genome (Lazy Load)...


In [3]:
print("[2/4] Parsing GTF...")
# Đọc GTF, lọc lấy Exon của Protein Coding Genes
df = pd.read_csv(GTF_FILE, sep='\t', comment='#', header=None, usecols=[0, 2, 3, 4, 6, 8],
                    names=['chrom', 'feature', 'start', 'end', 'strand', 'attribute'])
df = df[(df['feature'] == 'exon') & (df['attribute'].str.contains('gene_type "protein_coding"'))]
print(f"      -> {len(df)} exons found.")

[2/4] Parsing GTF...
      -> 2241203 exons found.


In [4]:
# Mở file CSV để ghi luồng (Stream Writing)
with open(OUTPUT_FILE, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'dna', 'label']) # Header

    # --- BƯỚC 3: XỬ LÝ POSITIVE (Labels 1 & 2) ---
    print("[3/4] Processing Positives...")
    seen_sites = set() # Set khử trùng lặp
    pos_buffer = []    # Bộ nhớ đệm tạm thời
    total_positives = 0
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Positives"):
        raw_chrom = row['chrom']
        norm_chrom = normalize_chrom(raw_chrom)
        if norm_chrom not in VALID_CHROMS: continue
        strand = row['strand']

        # Tính tọa độ tâm (Center at G)
        if strand == '+':
            donor_center = row['end'] + 1
            acc_center = row['start'] - 1
        else:
            donor_center = row['start'] - 1
            acc_center = row['end'] + 1

        # --- LABEL 1: DONOR (GT & GC) ---
        if (norm_chrom, donor_center, strand, 1) not in seen_sites:
            seq = get_sequence_strict(genome, raw_chrom, donor_center, strand, WINDOW)
            if seq:
                # Kiểm tra tâm: Chấp nhận GT và GC
                center_motif = seq[WINDOW : WINDOW+2]
                if center_motif in ['GT', 'GC']:
                    pos_buffer.append([f"Donor_{norm_chrom}_{donor_center}_{strand}", seq, 1])
                    seen_sites.add((norm_chrom, donor_center, strand, 1))
                    total_positives += 1

        # --- LABEL 2: ACCEPTOR (AG) ---
        if (norm_chrom, acc_center, strand, 2) not in seen_sites:
            seq = get_sequence_strict(genome, raw_chrom, acc_center, strand, WINDOW)
            if seq:
                # Kiểm tra tâm: Chấp nhận AG
                center_motif = seq[WINDOW-1 : WINDOW+1]
                if center_motif == 'AG':
                    pos_buffer.append([f"Acc_{norm_chrom}_{acc_center}_{strand}", seq, 2])
                    seen_sites.add((norm_chrom, acc_center, strand, 2))
                    total_positives += 1
        
        # Ghi vào đĩa khi buffer đầy (tránh tốn RAM)
        if len(pos_buffer) >= WRITE_BATCH_SIZE:
            writer.writerows(pos_buffer)
            pos_buffer = [] 

    # Ghi nốt phần dư
    if pos_buffer:
        writer.writerows(pos_buffer)
        pos_buffer = [] # Clear RAM hoàn toàn
        
    print(f"      -> Đã ghi {total_positives} mẫu Positive sạch.")
    
# --- BƯỚC 4: SINH DỮ LIỆU NEGATIVE KHỔNG LỒ (ĐÃ SỬA LỖI DUPLICATE) ---
    target_neg = total_positives * NEGATIVE_MULTIPLIER
    print(f"[4/4] Generating {target_neg} Negatives (Ratio 100:1:1)...")
    print("      Lưu ý: Bước này sẽ tốn RAM hơn để lưu set check trùng.")
    
    neg_count = 0
    neg_buffer = []
    
    # Iterator tuần hoàn
    df_sample = df.sample(frac=1).reset_index(drop=True)
    iter_rows = iter(df_sample.iterrows())
    
    pbar = tqdm(total=target_neg, desc="Mining Negatives")
    
    while neg_count < target_neg:
        try:
            _, row = next(iter_rows)
        except StopIteration:
            df_sample = df.sample(frac=1).reset_index(drop=True)
            iter_rows = iter(df_sample.iterrows())
            _, row = next(iter_rows)
        
        raw_chrom = row['chrom']
        norm_chrom = normalize_chrom(raw_chrom)
        if norm_chrom not in VALID_CHROMS: continue
        strand = row['strand']
        
        # Random vị trí
        rand_pos = random.randint(row['start'] - 10000, row['end'] + 10000)
        
        # --- [SỬA ĐỔI QUAN TRỌNG] ---
        # Tạo key đại diện cho mẫu Negative định lấy
        neg_key = (norm_chrom, rand_pos, strand, 0)
        
        # 1. Check trùng Positive (Site thật)
        if (norm_chrom, rand_pos, strand, 1) in seen_sites or \
           (norm_chrom, rand_pos, strand, 2) in seen_sites:
            continue

        # 2. Check trùng Negative đã lấy trước đó (FIX BUG DUPLICATE)
        if neg_key in seen_sites:
            continue
        # ----------------------------

        seq = get_sequence_strict(genome, raw_chrom, rand_pos, strand, WINDOW)
        if not seq: continue
        
        # Hard Negative Check
        center = seq[WINDOW]
        next_b = seq[WINDOW+1]
        prev_b = seq[WINDOW-1]
        
        is_fake_donor = (center == 'G' and next_b in ['T', 'C'])
        is_fake_acc   = (prev_b == 'A' and center == 'G')
        
        if is_fake_donor or is_fake_acc:
            neg_buffer.append([f"Neg_{norm_chrom}_{rand_pos}_{strand}", seq, 0])
            
            # --- [THÊM MỚI] Đánh dấu đã sử dụng ---
            seen_sites.add(neg_key) 
            # --------------------------------------

            neg_count += 1
            pbar.update(1)
            
            if len(neg_buffer) >= WRITE_BATCH_SIZE:
                writer.writerows(neg_buffer)
                neg_buffer = []

    # Ghi nốt phần cuối
    if neg_buffer:
        writer.writerows(neg_buffer)

print(f"✅ HOÀN TẤT! File saved at: {OUTPUT_FILE}")
print(f"      Dung lượng ước tính: ~9.5 GB")
print(f"      Tổng số dòng: {total_positives + neg_count}")
print(f"⚠️ QUAN TRỌNG: File này chưa được shuffle (Positive nằm đầu, Negative nằm cuối).")
print(f"   Hãy shuffle khi load vào model hoặc dùng lệnh Linux: 'shuf {OUTPUT_FILE} > shuffled.csv'")

[3/4] Processing Positives...


Positives: 100%|██████████| 2241203/2241203 [03:34<00:00, 10464.88it/s]


      -> Đã ghi 496576 mẫu Positive sạch.
[4/4] Generating 24828800 Negatives (Ratio 100:1:1)...
      Lưu ý: Bước này sẽ tốn RAM hơn để lưu set check trùng.


Mining Negatives: 100%|█████████▉| 24828773/24828800 [6:25:06<00:00, 942.85it/s]   

✅ HOÀN TẤT! File saved at: D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv
      Dung lượng ước tính: ~9.5 GB
      Tổng số dòng: 25325376
⚠️ QUAN TRỌNG: File này chưa được shuffle (Positive nằm đầu, Negative nằm cuối).
   Hãy shuffle khi load vào model hoặc dùng lệnh Linux: 'shuf D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv > shuffled.csv'


In [5]:
import csv
import os
from tqdm import tqdm

# --- CẤU HÌNH ---
TARGET_FILE = r'D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv'
TEMP_FILE = TARGET_FILE + '.tmp'
NEW_CHROM_COL = 'CHROM'

# Map đổi tên: {'Tên cũ': 'Tên mới'}
RENAME_MAP = {
    'label': 'Splicing_types',
    'dna': 'sequence'
}

print(f"🔄 Đang xử lý file: {TARGET_FILE}")
print(f"   - Đổi tên cột: {RENAME_MAP}")
print(f"   - Thêm/Update cột: {NEW_CHROM_COL}")

try:
    with open(TARGET_FILE, 'r', encoding='utf-8') as f_in, \
         open(TEMP_FILE, 'w', encoding='utf-8', newline='') as f_out:
        
        reader = csv.reader(f_in)
        writer = csv.writer(f_out)
        
        # 1. XỬ LÝ HEADER (Đổi tên & Thêm cột)
        try:
            original_header = next(reader)
        except StopIteration:
            print("❌ File rỗng!")
            raise Exception("File empty")
            
        # Bước A: Đổi tên các cột cũ (label -> Splicing_types, dna -> sequence)
        renamed_header = [RENAME_MAP.get(col, col) for col in original_header]
        
        # Bước B: Kiểm tra cột CHROM
        if NEW_CHROM_COL in renamed_header:
            print(f"⚠️ Cột '{NEW_CHROM_COL}' đã tồn tại -> Sẽ cập nhật dữ liệu.")
            final_header = renamed_header
            chrom_idx = renamed_header.index(NEW_CHROM_COL)
        else:
            final_header = renamed_header + [NEW_CHROM_COL]
            chrom_idx = None # Chưa có, sẽ append vào cuối
            
        writer.writerow(final_header)
        
        # 2. XỬ LÝ DỮ LIỆU (Streaming)
        for row in tqdm(reader, desc="Processing Rows", unit=" rows"):
            if not row: continue
            
            # Logic lấy Chromosome từ ID (Donor_1_65434_+)
            row_id = row[0]
            try:
                parts = row_id.split('_')
                if len(parts) > 1:
                    raw_chrom = parts[1]
                    # Format: chr1, chr2, chrX...
                    chrom_val = raw_chrom if raw_chrom.startswith('chr') else f"chr{raw_chrom}"
                else:
                    chrom_val = "unknown"
                
                # Logic ghi vào dòng
                if chrom_idx is not None:
                    # Nếu cột CHROM đã có sẵn (do chạy lại nhiều lần) -> Ghi đè
                    if len(row) > chrom_idx:
                        row[chrom_idx] = chrom_val
                    else:
                        row.append(chrom_val)
                    writer.writerow(row)
                else:
                    # Nếu chưa có -> Thêm mới vào cuối
                    writer.writerow(row + [chrom_val])
                    
            except Exception:
                # Fallback an toàn cho dòng lỗi
                writer.writerow(row + (['unknown'] if chrom_idx is None else []))

    # 3. Thay thế file cũ
    os.replace(TEMP_FILE, TARGET_FILE)
    print(f"✅ HOÀN TẤT! File đã được cập nhật.")
    print(f"   Header mới: {final_header}")

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file '{TARGET_FILE}'.")
except Exception as e:
    if os.path.exists(TEMP_FILE): os.remove(TEMP_FILE)
    print(f"❌ Lỗi Runtime: {e}")

🔄 Đang xử lý file: D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv
   - Đổi tên cột: {'label': 'Splicing_types', 'dna': 'sequence'}
   - Thêm/Update cột: CHROM


Processing Rows: 25325376 rows [08:53, 47475.31 rows/s]:25:19<00:00, 942.85it/s]


✅ HOÀN TẤT! File đã được cập nhật.
   Header mới: ['id', 'sequence', 'Splicing_types', 'CHROM']


In [6]:
import csv
import sys
import time
from collections import Counter
from tqdm import tqdm

# ================= CẤU HÌNH =================
# Sửa đường dẫn file nếu cần
FILE_PATH = r'D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv'
SAMPLE_ROWS = 5  # Số dòng in mẫu

def inspect_and_validate_full(file_path):
    print(f"🚀 BẮT ĐẦU KIỂM TRA FILE (CẤU TRÚC MỚI): {file_path}")
    
    start_time = time.time()
    
    # 1. BIẾN THỐNG KÊ
    stats = {
        "total_rows": 0,
        "null_counts": {},
        "seq_lengths": Counter(),      # Thống kê độ dài sequence
        "label_counts": Counter(),     # Thống kê Splicing_types
        "chrom_counts": Counter(),     # Thống kê CHROM (Mới)
        "samples": []
    }
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            
            # --- 2. ĐỌC & MAP HEADER ---
            try:
                header = next(reader)
                # Khởi tạo đếm null
                for col in header: stats["null_counts"][col] = 0
                
                # Tìm vị trí (index) của các cột quan trọng theo tên mới
                # Dùng next(..., None) để tránh lỗi nếu không tìm thấy cột
                idx_seq = next((i for i, c in enumerate(header) if c in ['sequence', 'dna']), None)
                idx_lbl = next((i for i, c in enumerate(header) if c in ['Splicing_types', 'label']), None)
                idx_chr = next((i for i, c in enumerate(header) if c in ['CHROM', 'chrom']), None)
                
                print(f"   -> Header tìm thấy: {header}")
                print(f"   -> Mapping index: Seq={idx_seq}, Label={idx_lbl}, Chrom={idx_chr}")
                
            except StopIteration:
                print("❌ LỖI: File rỗng!")
                return

            # --- 3. QUÉT DỮ LIỆU (SINGLE PASS) ---
            pbar = tqdm(reader, desc="Analyzing", unit=" rows")
            
            for i, row in enumerate(pbar):
                stats["total_rows"] += 1
                
                # A. LẤY MẪU
                if i < SAMPLE_ROWS:
                    stats["samples"].append(row)
                
                # B. CHECK NULL & VALIDATE
                for col_idx, val in enumerate(row):
                    if col_idx >= len(header): break
                    if not val.strip():
                        col_name = header[col_idx]
                        stats["null_counts"][col_name] += 1
                
                # C. THỐNG KÊ GIÁ TRỊ (Dựa trên index đã map)
                if idx_seq is not None and len(row) > idx_seq:
                    stats["seq_lengths"][len(row[idx_seq])] += 1
                    
                if idx_lbl is not None and len(row) > idx_lbl:
                    stats["label_counts"][row[idx_lbl]] += 1
                    
                if idx_chr is not None and len(row) > idx_chr:
                    stats["chrom_counts"][row[idx_chr]] += 1

    except FileNotFoundError:
        print(f"❌ Lỗi: Không tìm thấy file {file_path}")
        return

    # --- 4. BÁO CÁO KẾT QUẢ ---
    elapsed = time.time() - start_time
    total = stats["total_rows"]
    
    print("\n" + "="*80)
    print(f"📊 BÁO CÁO DỮ LIỆU (Time: {elapsed:.2f}s | Total: {total:,} rows)")
    print("="*80)

    # 4.1 Bảng mẫu (Cập nhật 4 cột)
    print(f"\n1. MẪU DỮ LIỆU ({SAMPLE_ROWS} dòng đầu):")
    # Format in: ID | Sequence | Type | Chrom
    print(f"   {'ID':<25} | {'Sequence (Preview)':<20} | {'Type':<10} | {'Chrom':<8}")
    print(f"   {'-'*25} | {'-'*20} | {'-'*10} | {'-'*8}")
    
    for row in stats["samples"]:
        # Lấy giá trị an toàn (phòng trường hợp dòng thiếu cột)
        _id = row[0] if len(row) > 0 else ""
        _seq = row[idx_seq] if idx_seq is not None and len(row) > idx_seq else "N/A"
        _lbl = row[idx_lbl] if idx_lbl is not None and len(row) > idx_lbl else "N/A"
        _chr = row[idx_chr] if idx_chr is not None and len(row) > idx_chr else "N/A"
        
        # Cắt ngắn sequence
        seq_show = _seq[:6] + "..." + _seq[-4:] if len(_seq) > 10 else _seq
        print(f"   {_id:<25} | {seq_show:<20} | {_lbl:<10} | {_chr:<8}")

    # 4.2 Báo cáo Null
    print(f"\n2. KIỂM TRA NULL (COMPLETENESS):")
    has_null = False
    print(f"   {'Tên Cột':<20} | {'Số Null':<10} | {'% Lỗi'}")
    for col, count in stats["null_counts"].items():
        pct = (count / total * 100) if total > 0 else 0
        status = "✅" if count == 0 else f"❌ {count:,}"
        print(f"   {col:<20} | {status:<10} | {pct:.4f}%")
        if count > 0: has_null = True
    if not has_null: print("   -> Sạch 100%.")

    # 4.3 Phân bố Label
    print(f"\n3. PHÂN BỐ TYPE (Splicing_types):")
    for lbl, count in stats["label_counts"].items():
        print(f"   - '{lbl}': {count:,} mẫu")

    # 4.4 Phân bố Chrom (Mới)
    print(f"\n4. PHÂN BỐ CHROMOSOME (Top 5):")
    # Chỉ in 5 chrom phổ biến nhất để đỡ dài dòng
    for ch, count in stats["chrom_counts"].most_common(5):
        print(f"   - {ch}: {count:,} mẫu")
    if len(stats["chrom_counts"]) > 5: print(f"   ... và {len(stats['chrom_counts'])-5} chrom khác.")

    # 4.5 Sequence Length
    print(f"\n5. SEQUENCE LENGTH:")
    for length, count in stats["seq_lengths"].items():
        status = "✅ Chuẩn" if length == 601 else "⚠️ LỆCH"
        print(f"   - Length {length}: {count:,} mẫu ({status})")

if __name__ == "__main__":
    inspect_and_validate_full(FILE_PATH)

🚀 BẮT ĐẦU KIỂM TRA FILE (CẤU TRÚC MỚI): D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv
   -> Header tìm thấy: ['id', 'sequence', 'Splicing_types', 'CHROM']
   -> Mapping index: Seq=1, Label=2, Chrom=3


Analyzing: 25325376 rows [03:36, 116985.01 rows/s]


📊 BÁO CÁO DỮ LIỆU (Time: 216.53s | Total: 25,325,376 rows)

1. MẪU DỮ LIỆU (5 dòng đầu):
   ID                        | Sequence (Preview)   | Type       | Chrom   
   ------------------------- | -------------------- | ---------- | --------
   Donor_1_65434_+           | AAAAGT...TTAA        | 1          | chr1    
   Donor_1_65574_+           | GATAGC...TTCC        | 1          | chr1    
   Acc_1_65519_+             | CTTTAT...CTCC        | 2          | chr1    
   Acc_1_69036_+             | AAAGGA...GCGC        | 2          | chr1    
   Donor_1_924949_+          | ACCTCA...GAGC        | 1          | chr1    

2. KIỂM TRA NULL (COMPLETENESS):
   Tên Cột              | Số Null    | % Lỗi
   id                   | ✅          | 0.0000%
   sequence             | ✅          | 0.0000%
   Splicing_types       | ✅          | 0.0000%
   CHROM                | ✅          | 0.0000%
   -> Sạch 100%.

3. PHÂN BỐ TYPE (Splicing_types):
   - '1': 250,830 mẫu
   - '2': 245,746 mẫu
   - '0': 24,82

In [7]:
import csv
import sys
import time
from collections import Counter
from tqdm import tqdm

# ================= CẤU HÌNH =================
FILE_PATH = r'D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv'
WINDOW = 300 # Bán kính như thiết lập ban đầu
CENTER_IDX = WINDOW # Vị trí tâm (300)

def validate_biological_data(file_path):
    print(f"🧬 BẮT ĐẦU KIỂM TRA LOGIC SINH HỌC: {file_path}")
    
    stats = {
        "total": 0,
        "invalid_chars": 0,   # Sequence chứa ký tự lạ
        "motif_errors": 0,    # Sai Motif (Donor k phải GT, Acc k phải AG)
        "label_dist": Counter(),
        "chrom_dist": Counter()
    }
    
    valid_bases = set("ACGTN")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            header = next(reader, None)
            
            # Map index cột (để code chạy đúng dù cột đổi chỗ)
            # Tìm cột theo tên mới hoặc cũ
            try:
                idx_seq = next(i for i, c in enumerate(header) if c in ['sequence', 'dna'])
                idx_lbl = next(i for i, c in enumerate(header) if c in ['Splicing_types', 'label'])
                idx_chr = next(i for i, c in enumerate(header) if c in ['CHROM', 'chrom'])
            except StopIteration:
                print("❌ Lỗi: Không tìm thấy đủ các cột sequence, label, CHROM!")
                return

            print(f"   -> Mapping OK: Seq={idx_seq}, Label={idx_lbl}, Chrom={idx_chr}")

            # QUÉT DỮ LIỆU
            for row in tqdm(reader, desc="Bio-Checking", unit=" rows"):
                if not row: continue
                stats["total"] += 1
                
                seq = row[idx_seq]
                label = row[idx_lbl]
                chrom = row[idx_chr]
                
                stats["label_dist"][label] += 1
                stats["chrom_dist"][chrom] += 1
                
                # 1. CHECK KÝ TỰ LẠ (Vocabulary)
                # Dùng set check cho nhanh
                if not set(seq).issubset(valid_bases):
                    stats["invalid_chars"] += 1

                # 2. CHECK MOTIF SINH HỌC (Quan trọng!)
                # Chỉ check nếu độ dài sequence đúng 601
                if len(seq) == (WINDOW * 2 + 1):
                    # Donor (Label 1): Tâm phải là GT hoặc GC
                    # Logic cắt: seq[300:302]
                    if label == '1':
                        motif = seq[CENTER_IDX : CENTER_IDX+2]
                        if motif not in ['GT', 'GC']:
                            stats["motif_errors"] += 1
                            
                    # Acceptor (Label 2): Tâm phải là AG
                    # Logic cắt: seq[299:301] (Vì AG nằm ngay trước exon)
                    elif label == '2':
                        motif = seq[CENTER_IDX-1 : CENTER_IDX+1]
                        if motif != 'AG':
                            stats["motif_errors"] += 1

    except Exception as e:
        print(f"❌ Lỗi Runtime: {e}")
        return

    # BÁO CÁO
    total = stats["total"]
    print("\n" + "="*60)
    print(f"🔬 KẾT QUẢ KIỂM TRA SINH HỌC ({total:,} mẫu)")
    print("="*60)
    
    # 1. Ký tự lạ
    status_char = "✅ Sạch" if stats["invalid_chars"] == 0 else f"❌ {stats['invalid_chars']:,} mẫu lỗi"
    print(f"1. TỪ VỰNG DNA (Chỉ A,C,G,T,N): {status_char}")
    
    # 2. Motif
    # Lưu ý: Nếu Motif lỗi nhiều -> Code sinh dữ liệu bị lệch index slicing
    pct_motif = (stats["motif_errors"] / total * 100)
    status_motif = "✅ Chuẩn xác" if pct_motif < 1.0 else f"⚠️ ĐÁNG NGỜ ({stats['motif_errors']:,} mẫu sai motif)"
    print(f"2. MOTIF CONSISTENCY (GT/AG): {status_motif}")
    if stats["motif_errors"] > 0:
        print("   (Nếu số lượng lỗi motif quá lớn, hãy kiểm tra lại logic cắt chuỗi!)")

    # 3. Chromosome Check
    print(f"3. CHROMOSOME COVERAGE:")
    print(f"   - Tìm thấy {len(stats['chrom_dist'])} nhiễm sắc thể.")
    # In cảnh báo nếu thiếu chr1 (thường là do file GTF lỗi hoặc filter sai)
    if 'chr1' not in stats['chrom_dist']:
        print("   ⚠️ CẢNH BÁO: Không thấy dữ liệu từ chr1!")

if __name__ == "__main__":
    validate_biological_data(FILE_PATH)

🧬 BẮT ĐẦU KIỂM TRA LOGIC SINH HỌC: D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv
   -> Mapping OK: Seq=1, Label=2, Chrom=3


Bio-Checking: 25325376 rows [05:15, 80376.29 rows/s]


🔬 KẾT QUẢ KIỂM TRA SINH HỌC (25,325,376 mẫu)
1. TỪ VỰNG DNA (Chỉ A,C,G,T,N): ✅ Sạch
2. MOTIF CONSISTENCY (GT/AG): ✅ Chuẩn xác
3. CHROMOSOME COVERAGE:
   - Tìm thấy 24 nhiễm sắc thể.


In [8]:
import csv
import sys
import time
from collections import Counter
from tqdm import tqdm

# CẤU HÌNH FILE
FILE_PATH = r'D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv'
MAX_RAM_IDS = 30_000_000  # Giới hạn lưu 30 triệu ID để tránh tràn RAM

def check_data_stats(file_path):
    print(f"🚀 BẮT ĐẦU KIỂM TRA FILE: {file_path}")
    
    # 1. Khởi tạo biến đếm
    stats = {
        "total_rows": 0,
        "null_rows": 0,       # Dòng có giá trị rỗng
        "duplicate_ids": 0,   # ID bị trùng
    }
    label_counts = Counter()  # Đếm Value Count cho Label
    seen_ids = set()          # Set lưu ID để check trùng
    check_dup = True          # Cờ bật/tắt check trùng (bảo vệ RAM)

    start_time = time.time()

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            header = next(reader, None) # Bỏ qua header
            
            # Streaming Loop (Đọc từng dòng -> Xử lý -> Xóa khỏi RAM)
            for row in tqdm(reader, desc="Scanning", unit=" rows"):
                stats["total_rows"] += 1
                
                # Check cấu trúc cơ bản (để tránh lỗi index)
                if len(row) < 3: continue 
                _id, _dna, _label = row[0], row[1], row[2]

                # --- 1. CHECK NULL ---
                if not _id.strip() or not _dna.strip() or not _label.strip():
                    stats["null_rows"] += 1

                # --- 2. CHECK DUPLICATE ID ---
                if check_dup:
                    if _id in seen_ids:
                        stats["duplicate_ids"] += 1
                    else:
                        seen_ids.add(_id)
                    
                    # Cơ chế an toàn: Tự ngắt check lặp nếu ID quá nhiều (>30 triệu)
                    if len(seen_ids) > MAX_RAM_IDS:
                        print("⚠️ RAM Warning: Tạm dừng check Duplicate để bảo vệ máy.")
                        check_dup = False
                        seen_ids.clear() # Giải phóng RAM

                # --- 3. VALUE COUNT (Đếm Label) ---
                label_counts[_label] += 1

    except FileNotFoundError:
        print("❌ Lỗi: Không tìm thấy file csv.")
        return

    # --- 4. THỐNG KÊ CLASS (BÁO CÁO) ---
    total = stats["total_rows"]
    elapsed = time.time() - start_time

    print("\n" + "="*40)
    print(f"📊 BẢNG THỐNG KÊ DỮ LIỆU (Time: {elapsed:.2f}s)")
    print("="*40)
    
    print(f"1. TỔNG QUAN:")
    print(f"   - Tổng số dòng: {total:,}")
    print(f"   - Null rows   : {stats['null_rows']:,}  ({(stats['null_rows']/total*100):.4f}%)")
    print(f"   - Duplicate ID: {stats['duplicate_ids']:,}")

    print(f"\n2. VALUE COUNT & CLASS STATISTICS:")
    print(f"   {'Label':<10} | {'Count':<15} | {'Percentage':<10}")
    print(f"   {'-'*10} | {'-'*15} | {'-'*10}")
    
    # Sắp xếp label theo thứ tự 0, 1, 2
    for label in sorted(label_counts.keys()):
        count = label_counts[label]
        percent = (count / total * 100) if total > 0 else 0
        
        # Mapping tên class cho dễ hiểu
        label_name = label
        if label == '0': label_name = "0 (Neg)"
        elif label == '1': label_name = "1 (Donor)"
        elif label == '2': label_name = "2 (Acc)"
            
        print(f"   {label_name:<10} | {count:<15,} | {percent:.4f}%")

    print("="*40)

    # Đánh giá nhanh
    if stats['null_rows'] == 0 and stats['duplicate_ids'] == 0:
        print("✅ Dữ liệu SẠCH (Không Null, Không Trùng).")
    else:
        print("⚠️ Dữ liệu CÓ VẤN ĐỀ. Vui lòng kiểm tra lại logic sinh mẫu.")

if __name__ == "__main__":
    check_data_stats(FILE_PATH)

🚀 BẮT ĐẦU KIỂM TRA FILE: D:\Study\5-FA25\AiTa_Lab_Research\Sequence_GENCODE\pre_train_splicing_prediction.csv


Scanning: 25325376 rows [03:28, 121466.89 rows/s]



📊 BẢNG THỐNG KÊ DỮ LIỆU (Time: 208.51s)
1. TỔNG QUAN:
   - Tổng số dòng: 25,325,376
   - Null rows   : 0  (0.0000%)
   - Duplicate ID: 0

2. VALUE COUNT & CLASS STATISTICS:
   Label      | Count           | Percentage
   ---------- | --------------- | ----------
   0 (Neg)    | 24,828,800      | 98.0392%
   1 (Donor)  | 250,830         | 0.9904%
   2 (Acc)    | 245,746         | 0.9704%
✅ Dữ liệu SẠCH (Không Null, Không Trùng).
